In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup,NavigableString, Tag
import time
import requests
import re
import csv
from selenium.webdriver.common.by import By
import pandas as pd
import os

driver = webdriver.Chrome()
    
driver.get("https://roo.cash/creditcard?source=google&medium=ad_CC_kw_g&content=kw_cc") #自動打開網頁（不要關）
    
time.sleep(5) #暫停
    
soup = BeautifulSoup(driver.page_source, 'html.parser') #把script變成美麗湯可讀
#print(soup.prettify())  #輸出排版後的HTML內容

ranking_name = [] #排行榜名稱
ranking = [] #連結相關（英文）

results = soup.find_all('div', class_='tab carousel-cell text-NRooBlue-95 text-md font-md leading-6 rounded-full border border-NRooBlue-95 hover:bg-NRooBlue-10 mt-4 mr-3 lg:mt-3 lg:mr-3 py-1 px-5 whitespace-nowrap cursor-pointer')
for result in results:
    e_name = result.get('id')
    name = result.text
    print(name)
    print(e_name[4:])
    ranking_name.append(name)
    ranking.append(e_name[4:])

card_name_all = [] #全站卡名
card_link_all = [] #全站連結
card_img_all = [] #全站圖片
for i in range(len(ranking)):
    
    if i == 0: #即時排行網址特殊
        driver = webdriver.Chrome()
        driver.get("https://roo.cash/creditcard/recommendation?source=google&medium=ad_CC_kw_g&content=kw_cc") #自動打開網頁（不要關）
        time.sleep(5) #暫停
        soup = BeautifulSoup(driver.page_source, 'html.parser') #把script變成美麗湯可讀
        result = soup.find('div', id='plans')  #找卡名
        titles = result.find_all("h3")
        card_name= []
        for title in titles:
            #print('☆'+title.text, end="\n")
            card_name.append(title.text)
            card_name_all.append(title.text)
            
        card_link =[] #存卡片連結
        cards = result.find_all("div", class_='plansShadow')
        for card in cards:
            card_link.append(card.find('a').get('href'))
            card_link_all.append(card.find('a').get('href'))
            
        #for i in range(len(card_link)):
            #print(card_link[i])  
            
        card_img = [] #卡圖片
        results = soup.find_all('div', class_='w-20 h-20 inline-block mr-3 lg:mr-5 flex-none')
        for result in results:
            #print(result.find('img').get('src'))
            card_img.append(result.find('img').get('src'))
            card_img_all.append(result.find('img').get('src'))
        
        #card_dict = {
        #    '卡名':card_name,
        #    '卡連結':card_link,
        #    '卡圖片':card_img
        #}
        ##path = '/Users/takoyaki/Downloads/專題爬蟲/袋鼠csv/排名csv/'    
        #df = pd.DataFrame(card_dict)
        #folder_name = '排名csv'
        #path = "/Users/takoyaki/Downloads/專題爬蟲/袋鼠csv"  
    #
        #folder_path = os.path.join(path, folder_name)
        #if not os.path.isdir(folder_path):
        #    os.makedirs(folder_path, mode=0o777)
        #path0 = str(path+'/'+folder_name+"/")
        #df.to_csv(path0+"袋鼠"+ranking_name[i]+".csv", index=False) 
        
    else:
        driver = webdriver.Chrome()
        driver.get("https://roo.cash/creditcard/ranking/"+ranking[i]+"?source=google&medium=ad_CC_kw_g&content=kw_cc") #自動打開網頁（不要關）
        time.sleep(5) #暫停
        soup = BeautifulSoup(driver.page_source, 'html.parser') #把script變成美麗湯可讀
        result = soup.find('div', id='plans')  #找卡名
        titles = result.find_all("h3")
        card_name= []
        for title in titles:
            #print('☆'+title.text, end="\n")
            card_name.append(title.text)
            card_name_all.append(title.text)
            
        card_link =[] #存卡片連結
        cards = result.find_all("div", class_='plansShadow')
        for card in cards:
            card_link.append(card.find('a').get('href'))
            card_link_all.append(card.find('a').get('href'))
        #for i in range(len(card_link)):
            #print(card_link[i])  
            
        card_img = [] #卡圖片
        results = soup.find_all('div', class_='w-20 h-20 inline-block mr-3 lg:mr-5 flex-none')
        for result in results:
            #print(result.find('img').get('src'))
            card_img.append(result.find('img').get('src'))
            card_img_all.append(result.find('img').get('src'))
            
        #card_dict = {
        #    '卡名':card_name,
        #    '卡連結':card_link,
        #    '卡圖片':card_img
        #}
        ##path = '/Users/takoyaki/Downloads/專題爬蟲/袋鼠csv/排名csv/' 
        #
        #df = pd.DataFrame(card_dict)
        #folder_name = '排名csv'
        #path = "/Users/takoyaki/Downloads/專題爬蟲/袋鼠csv"  
    #
        #folder_path = os.path.join(path, folder_name)
        #if not os.path.isdir(folder_path):
        #    os.makedirs(folder_path, mode=0o777)
        #path0 = str(path+'/'+folder_name+"/")
        #df.to_csv(path0+"袋鼠"+ranking_name[i]+".csv", index=False) 
     
              
driver.close()

card_apply_link =[] #申請連結
card_fee = [] #年費下標
card_fee1 = [] #年費內文
card_rate = [] #利率下標
card_rate1 = [] #利率內文
card_condition = [] #條件下標
card_condition1 = [] #條件內文

for i in range(len(card_link_all)): #卡連結
    headers = {'content-type': 'text/html; charset=UTF-8','user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36(KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'
}
    response = requests.get(card_link_all[i], headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    
    space＿f = ['一、', '二、', '三、'] #大標
    title_f = ['1. ','2. ','3. ','4. ','5.'] #中標
    
    result = soup.find('h2',class_='text-NRooBlue-100 text-2xl md:text-3xl text-center font-bold') #卡名
    print()
    print('☆'+result.text+'☆')
    print()
    
    space_list = [] #年費、特色
    title_list = [] #年費、特色
    space1_text_final = [] #內文
    
    spaces = soup.find_all('div', class_ = 'flex text-md md:text-lg align-top mb-3')
    for space in spaces:
        space_list.append(space)
        
    for j in range(len(space_list)):
        title = space_list[j].find('div', class_='text-NRooBlue-100 mr-4 whitespace-nowrap') #大標
        title_list.append(title.text)
        text_list = []
        print(space_f[j] + title_list[j], end='\n')
        print()
        results = space_list[j].find_all('div', class_ = 'text-NRooBlue-100 font-bold') #內文
        for result in results:
            text_list.append(result.text)
        text = '\n'.join(text_list)
        space1_text_final.append(text)
        for k in range(len(text_list)):
            print(title_f[k]+text_list[k], end= '\n')
        
    result = soup.find('div',onclick="openlink(event);").get('data-link')
    print('\n立即申請: \n'+str(result)+'\n')
    card_apply_link.append(str(result)) #申請連結寫入list
            
    
    s = Service(r"/usr/local/bin/chromedriver") #使用chromedriver、selenium
    driver = webdriver.Chrome(service=s)
    
    driver.get(card_link_all[i]) #自動打開網頁（不要關）
    
    time.sleep(3) #暫停
    
    soup = BeautifulSoup(driver.page_source, 'html.parser') #把script變成美麗湯可讀
    
    search = soup.find('h2', class_ = 'text-NRooBlue-100 text-2xl md:text-3xl font-bold pb-1 md:pb-2')
    print('☆'+search.text+'☆') #大大標（優惠與權益）
    
    space_list = ['basic-offers mb-3','new-user-offers mb-3','responsibilities-and-rights'] #總共三區
    
    
    for j in range(len(space_list)):
        space = soup.find('div',class_=space_list[j]) #區
        if space == None : #如果有少，跳過
            continue
            
       
    #存入 
    
    #大標（一、二）
        title1 = space.find('h2', class_ = 'font-bold text-2lg md:text-xl text-NRooBlue-100 py-4 border-b-1 border-NRooGrey-60')
    #中標（1.、2.)
        title_list1 = [] 
        titles = space.find_all('div', class_ = 'offer-title mx-3 md:mx-5 text-NRooBlue-100 text-left flex-1')
        for title in titles:
            title_list1.append(title)
    #內文        
        text_list = [] 
        texts = space.find_all('span', class_ = 'text-NRooBlue-100 pb-5')
        for text in texts:
            text_list.append(text)
        
    
    #印
        print()
        print(space_f[j] + title1.text) #大標
    
        for l in range(len(title_list1)): #中標
            print()
            result = title_list1[l].find('h3')
            print(title_f[l] + result.text)
            title_list.append(result.text)
            result = title_list1[l].find('h4') #下標
            print('   '+result.text)
            
            print()
            text2 = []
            results = text_list[l].find_all('p') #內文
            for result in results:
                print(result.text, end='\n')
                text2.append(result.text)
            text = '\n'.join(text2)
            space1_text_final.append(text)
                
        if title1.text =='卡片須知':
            for l in range(len(title_list1)): #中標
                result1 = title_list1[l].find('h3')
                result = title_list1[l].find('h4') #下標
                if result1.text == '年費':
                    card_fee.append(result.text) #存入下標
                    results = text_list[l].find_all('p') #內文
                    card_text33 = []
                    for result in results:
                        card_text33.append(result.text)
                    card_fee1.append('\n'.join(card_text33)) #存入內文
                elif result1.text == '循環利率':
                    card_rate.append(result.text) #存入下標
                    results = text_list[l].find_all('p') #內文
                    card_text33 = []
                    for result in results:
                        card_text33.append(result.text)
                    card_rate1.append('\n'.join(card_text33)) #存入內文
                elif result1.text == '申辦條件':
                    card_condition.append(result.text) #存入下標
                    results = text_list[l].find_all('p') #內文
                    card_text33 = []
                    for result in results:
                        card_text33.append(result.text)
                    card_condition1.append('\n'.join(card_text33)) #存入內文
                else:
                    continue
                    
    card_dict1 = {
        '標題':title_list,
        '詳情':space1_text_final
    }
    #path = '/Users/takoyaki/Downloads/專題爬蟲/袋鼠csv/卡優惠csv/'    
    df = pd.DataFrame(card_dict1)
    folder_name = '卡優惠csv'
    path = "/Users/takoyaki/Downloads/專題爬蟲/袋鼠csv"  

    folder_path = os.path.join(path, folder_name)
    if not os.path.isdir(folder_path):
        os.makedirs(folder_path, mode=0o777)
    path0 = str(path+'/'+folder_name+"/")
    df.to_csv(path0+"袋鼠"+card_name_all[i]+".csv", index=False)
    
    driver.close()

    
card_dict = {
            '卡名':card_name_all
            #'卡連結':card_link_all,
            #'卡圖片':card_img_all
        }
#card_dict['申請連結'] = card_apply_link  #申請連結
card_dict['年費'] = card_fee #年費下標
card_dict['年費詳細敘述'] = card_fee1 #年費內文
#card_dict['循環利率'] = card_rate #利率下標
card_dict['循環利率詳細敘述'] = card_rate1 #利率內文
card_dict['申請條件'] = card_condition #條件下標
card_dict['申請條件詳細敘述'] = card_condition1 #條件內文

#path = '/Users/takoyaki/Downloads/專題爬蟲/袋鼠csv/卡csv/'   
df = pd.DataFrame(card_dict)
folder_name = '卡csv'
path = "/Users/takoyaki/Downloads/專題爬蟲/袋鼠csv"  
folder_path = os.path.join(path, folder_name)
if not os.path.isdir(folder_path):
    os.makedirs(folder_path, mode=0o777)
path0 = str(path+'/'+folder_name+"/")
df.to_csv(path0+"袋鼠全站.csv", index=False)